In [1]:
import re
import pandas as pd 
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Functions for detecting glaucoma implants from free text operative notes

In [27]:
def gddmodelfeatures(text): 
    if len(re.findall('(?i)Baerveldt', text))>0: 
        gddtype="Baerveldt"
        try: gddmodel=re.findall('(?i)Baerveldt.*?([23]50)', text)[0]
        except: gddmodel=None        
    elif len(re.findall('(?i)Ahmed', text))>0: 
        gddtype="Ahmed"
        try: gddmodel=re.findall('(?i)Ahmed.*?(FP[7|8]|S[2|3])', text)[0]
        except: gddmodel=None
    elif len(re.findall('(?i)Molteno', text))>0: 
        gddtype="Molteno"
        try: gddmodel=re.findall('(?i)Molteno.*?([SDMG][1S])', text)[0]
        except: gddmodel=None
    elif len(re.findall('(?i)iStent', text))>0: 
        gddtype="iStent"
        try: gddmodel=re.findall('(?i)iStent\s*(inject)', text)[0]
        except: gddmodel=None
    elif len(re.findall('(?i)CyPass', text))>0: 
        gddtype="CyPass"
        gddmodel=None
    elif len(re.findall('(?i)Xen', text))>0: 
        gddtype="Xen"
        gddmodel=None
    elif len(re.findall('(?i)(Ex-Press|express(\s.*)shunt)', text))>0: 
        gddtype="Ex-PRESS"
        gddmodel=None
    elif len(re.findall('(?i)Hydrus', text))>0: 
        gddtype = "Hydrus"
        gddmodel=None
    else: 
        gddtype=None
        gddmodel=None
    return (gddtype, gddmodel)

#at the time this was initially developed, Hydrus was not yet on the market. Since then the function was updated to include this. 
#At the time this was initially developed, iStent Inject was not yet on the market. Since then the function was updated to include this. 

# Examples

## Example simple code

In [29]:
#example simple code for how this works 
gddmodelfeatures("Ahmed fp7")
gddmodelfeatures("ex-press minishunt")
gddmodelfeatures("expressed minishunt")
gddmodelfeatures("Hydrus")
gddmodelfeatures("iStent")
gddmodelfeatures("iStent inject")
gddmodelfeatures("iStent was implanted etc. etc. etc. moxifloxacin was injected")

('Ahmed', 'fp7')

('Ex-PRESS', None)

(None, None)

('Hydrus', None)

('iStent', None)

('iStent', 'inject')

('iStent', None)

## Example code for how this might work on a pandas dataframe containing a column of operative reports

In [33]:
#example code for how this might work on a pandas dataframe containing a column of operative reports 
df=pd.DataFrame(["Ahmed FP7", "Baerveldt", "Baerveldt 350", "Cypass", "Express shunt", "iStent", "Xen", "iStent Inject", "Hydrus"], columns=["note"])
df

,note
0,Ahmed FP7
1,Baerveldt
2,Baerveldt 350
3,Cypass
4,Express shunt
5,iStent
6,Xen
7,iStent Inject
8,Hydrus


In [34]:
df["gddtype"], df["gddmodel"]=zip(*df["note"].apply(gddmodelfeatures))
df

,note,gddtype,gddmodel
0,Ahmed FP7,Ahmed,FP7
1,Baerveldt,Baerveldt,None
2,Baerveldt 350,Baerveldt,350
3,Cypass,CyPass,None
4,Express shunt,Ex-PRESS,None
5,iStent,iStent,None
6,Xen,Xen,None
7,iStent Inject,iStent,Inject
8,Hydrus,Hydrus,None
